# 대한민국 도시 고유 ID 주기
## Json파일을 이용해서 Folium으로 그리든, Cartogram으로 표현하든 두 방법 다 각 지역을 대표하는 고유 ID를 만들어줘야 한다.

In [2]:
import pandas as pd
import numpy as np

In [3]:
pop = pd.read_csv('./result/성별, 연령별 인구 - 시도(2019).csv',sep=',',encoding='utf8')
pop.head()

,광역시도,시도,20~39세계,20~39세남자,20~39세여자,65세 이상계,65세 이상남자,65세 이상여자,인구수계,인구수남자,인구수여자,소멸 비율,소멸 위기 지역
0,강원,강릉시,46008,25015,20993,40497,17378,23119,203057,101154,101903,0.518384,False
1,강원,고성군,4006,2307,1699,7005,2957,4048,23621,11933,11688,0.242541,True
2,강원,동해시,16311,8801,7510,16152,7075,9077,83888,42093,41795,0.464958,True
3,강원,삼척시,12428,7148,5280,14953,6314,8639,61833,31630,30203,0.353106,True
4,강원,속초시,15928,8517,7411,14010,5866,8144,76302,37488,38814,0.528979,False


In [18]:
pop['광역시도'].unique()

array(['강원', '경기', '경남', '경북', '고양시', '광주', '대구', '대전', '부산', '서울', '성남시',
       '세종', '수원시', '안산시', '안양시', '용인시', '울산', '인천', '전남', '전북', '전주시',
       '제주', '창원시', '천안시', '청주시', '충남', '충북', '포항시'], dtype=object)

## 각 지역 고유 ID를 만들기 전에 지역 이름이 겹치는 게 있는지 먼저 확인
- pop\['시도'\].unique() 는 인천 서구, 부산 서구 처럼 겹치는 게 있어서 250으로 안 나옴

In [4]:
len(pop['시도'])

250

In [5]:
pop['시도'][pop['시도'].str.find('분당') > 0 ]

Series([], Name: 시도, dtype: object)

## '광역시도' 값과 '시도' 값을 합치면 고유 ID (서울 강남, 인천 부평)을 만들 수 있다.
- 인천, 부산 등 광역시의 구(자치구) 도 있으나, 안양시나 수원시에 있는 구(행정구) 도 있다.
- 광역시도의 자치구는 '서울 중구' 처럼 나오긴 하지만, 행정구의 경우는 '안양시 동안구'가 되지 않고 '경기도 동안구'가 됨. 이걸 어떻게 해결할 수 있을까?

## 먼저 광역시가 아니면서 구를 가지고 있는 시와 그 행정구를 Dictionary로 만듦

In [6]:
si_name = [None] * len(pop) # 250
si_name[:5]

[None, None, None, None, None]

In [7]:
# 요게 제대로 안 들어감
tmp_gu_dict = {
    '수원':['장안구', '권선구', '팔달구', '영통구'], 
    '성남':['수정구', '중원구', '분당구'], 
    '안양':['만안구', '동안구'], 
    '안산':['상록구', '단원구'], 
    '고양':['덕양구', '일산동구', '일산서구'], 
    '용인':['처인구', '기흥구', '수지구'], 
    '청주':['상당구', '서원구', '흥덕구', '청원구'], 
    '천안':['동남구', '서북구'], 
    '전주':['완산구', '덕진구'], 
    '포항':['남구', '북구'], 
    '창원':['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
}

In [8]:
length = 0
for values in tmp_gu_dict.values():
    for elm in values:
        length += 1
length

32

1. 광역시도에 있는 이름의 끝 세자가 '광역시','특별시','자치시'로 끝나지 않으면 일반 시 혹은 군으로 처리
2. 강원도 고성군과 경상남도 고성군 처리
3. 세종특별자치시는 세종으로
4. 광역시도에서 앞 두 글자(서울특별시 => 서울), 나머지는 광역시도에서 앞 두 글자(서울특별시)와 시도에서 두 글자인 경우 모두, 아니면 앞 두 글자만 선택하며 고유 ID를 만듦

In [9]:
""" for i in pop.index:
    if pop['광역시도'][i] in ['서울','부산','대구','인천','광주','대전','울산']:
        if len(pop['시도'][i]) == 3:
            si_name[i] = pop['광역시도'][i] + ' ' + pop['시도'][i][1:]
        else:
            si_name[i] = pop['광역시도'][i] + ' ' + pop['시도'][i][1:-1]


    else:
        if pop['시도'][i][1:-1] == '고성' and pop['광역시도'][i] == '강원':
            si_name[i] = '고성(강원)'
        elif pop['시도'][i][:-1] == '고성' and pop['광역시도'][i] == '경남':
                si_name[i] = '고성(경남)'
        else:
            si_name[i] = pop['시도'][i][1:-1]        
        
            for keys, values in tmp_gu_dict.items():
                if pop['시도'][i] in values:
                    if len(pop['시도'][i]) == 2:
                        si_name[i] = keys + ' ' + pop['시도'][i]
                    elif pop['시도'][i] in ['마산합포구', '마산회원구']:
                        si_name[i] = keys + ' ' + pop['시도'][i][2:-1]
                    else:
                        si_name[i] = keys + ' ' + pop['시도'][i][:-1] """

" for i in pop.index:\n    if pop['광역시도'][i] in ['서울','부산','대구','인천','광주','대전','울산']:\n        if len(pop['시도'][i]) == 3:\n            si_name[i] = pop['광역시도'][i] + ' ' + pop['시도'][i][1:]\n        else:\n            si_name[i] = pop['광역시도'][i] + ' ' + pop['시도'][i][1:-1]\n\n\n    else:\n        if pop['시도'][i][1:-1] == '고성' and pop['광역시도'][i] == '강원':\n            si_name[i] = '고성(강원)'\n        elif pop['시도'][i][:-1] == '고성' and pop['광역시도'][i] == '경남':\n                si_name[i] = '고성(경남)'\n        else:\n            si_name[i] = pop['시도'][i][1:-1]        \n        \n            for keys, values in tmp_gu_dict.items():\n                if pop['시도'][i] in values:\n                    if len(pop['시도'][i]) == 2:\n                        si_name[i] = keys + ' ' + pop['시도'][i]\n                    elif pop['시도'][i] in ['마산합포구', '마산회원구']:\n                        si_name[i] = keys + ' ' + pop['시도'][i][2:-1]\n                    else:\n                        si_name[i] = keys + ' ' + pop['시도

# 구 이름에 앞에 띄어쓰기가 들어가 있어서 안 됐던거임. 어이없게...
# 앞으로 시작할 때 DF 전체에 strip()을 걸 수 있는 방법이 없나?

In [10]:
metro_list = ['서울', '부산', '대구', '인천', '대전', '광주', '울산']

for i in pop.index:
    if pop['광역시도'][i] in metro_list:
        if len(pop['시도'][i]) == 2:
            si_name[i] = pop['광역시도'][i] + ' ' + pop['시도'][i]
        else:
            si_name[i] = pop['광역시도'][i] + ' ' + pop['시도'][i][:-1]
    else:
        if pop['시도'][i][:-1] == '고성':
            if pop['광역시도'][i] == '강원':
                si_name[i] = '고성(강원)'
            else:
                si_name[i] = '고성(경남)'
        else:
            si_name[i] = pop['시도'][i][:-1]

        for key, values in tmp_gu_dict.items():
            if pop['시도'][i] in values:
                if len(pop['시도'][i]) == 2:
                    si_name[i] = key + ' ' + pop['시도'][i]
                elif pop['시도'][i] in ['마산합포구', '마산회원구']:
                    print(2, key, pop['시도'][i][2:-1])
                    si_name[i] = key + ' ' + pop['시도'][i][2:-1]
                else:
                    si_name[i] = key + ' ' + pop['시도'][i][:-1]

2 창원 합포
2 창원 회원


In [11]:
len(si_name)

250

In [12]:
si_name

['강릉',
 '고성(강원)',
 '동해',
 '삼척',
 '속초',
 '양구',
 '양양',
 '영월',
 '원주',
 '인제',
 '정선',
 '철원',
 '춘천',
 '태백',
 '평창',
 '홍천',
 '화천',
 '횡성',
 '가평',
 '과천',
 '광명',
 '광주',
 '구리',
 '군포',
 '김포',
 '남양주',
 '동두천',
 '부천',
 '시흥',
 '안성',
 '양주',
 '양평',
 '여주',
 '연천',
 '오산',
 '의왕',
 '의정부',
 '이천',
 '파주',
 '평택',
 '포천',
 '하남',
 '화성',
 '거제',
 '거창',
 '고성(경남)',
 '김해',
 '남해',
 '밀양',
 '사천',
 '산청',
 '양산',
 '의령',
 '진주',
 '창녕',
 '통영',
 '하동',
 '함안',
 '함양',
 '합천',
 '경산',
 '경주',
 '고령',
 '구미',
 '군위',
 '김천',
 '문경',
 '봉화',
 '상주',
 '성주',
 '안동',
 '영덕',
 '영양',
 '영주',
 '영천',
 '예천',
 '울릉',
 '울진',
 '의성',
 '청도',
 '청송',
 '칠곡',
 '고양 덕양',
 '고양 일산동',
 '고양 일산서',
 '광주 광산',
 '광주 남구',
 '광주 동구',
 '광주 북구',
 '광주 서구',
 '대구 남구',
 '대구 달서',
 '대구 달성',
 '대구 동구',
 '대구 북구',
 '대구 서구',
 '대구 수성',
 '대구 중구',
 '대전 대덕',
 '대전 동구',
 '대전 서구',
 '대전 유성',
 '대전 중구',
 '부산 강서',
 '부산 금정',
 '부산 기장',
 '부산 남구',
 '부산 동구',
 '부산 동래',
 '부산 부산진',
 '부산 북구',
 '부산 사상',
 '부산 사하',
 '부산 서구',
 '부산 수영',
 '부산 연제',
 '부산 영도',
 '부산 중구',
 '부산 해운대',
 '서울 강남',
 '서울 강동',
 '서울 강북',
 '서울 강서',
 

In [13]:
si_name = pd.Series(si_name)
# 분당이 3번째에 있는데 왜 여긴 안 나오지?
si_name[si_name.str.find( sub = '분당') > 0]

144    성남 분당
dtype: object

In [14]:
pop['ID'] = si_name

In [15]:
del pop['65세 이상남자']
del pop['65세 이상여자']
del pop['20~39세남자']

In [16]:
pop.head()

,광역시도,시도,20~39세계,20~39세여자,65세 이상계,인구수계,인구수남자,인구수여자,소멸 비율,소멸 위기 지역,ID
0,강원,강릉시,46008,20993,40497,203057,101154,101903,0.518384,False,강릉
1,강원,고성군,4006,1699,7005,23621,11933,11688,0.242541,True,고성(강원)
2,강원,동해시,16311,7510,16152,83888,42093,41795,0.464958,True,동해
3,강원,삼척시,12428,5280,14953,61833,31630,30203,0.353106,True,삼척
4,강원,속초시,15928,7411,14010,76302,37488,38814,0.528979,False,속초


In [17]:
pop.to_csv('./result/소멸 위기 지역.csv', sep=',', encoding = 'utf8')